In [2]:
library('DBI')
library('RPostgreSQL')
library('dbplyr')
library('dplyr')


Attaching package: ‘dplyr’


The following objects are masked from ‘package:dbplyr’:

    ident, sql


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [3]:
con <- DBI::dbConnect(
	PostgreSQL(),
	host="localhost",
	port=5432,
	dbname="knock100",
	user="guest",
	password="guest"
)

In [4]:
# get the list of tables available
DBI::dbListTables(con)

[1] "customer" "category" "geocode"  "product"  "receipt"  "store"

In [5]:
# define table name variables 
receipt_tbl <-  tbl(con,'receipt')
customer_tbl <-  tbl(con,'customer')
store_tbl <-  tbl(con, 'store')

In [6]:
receipt_tbl %>% count()

# Source:   lazy query [?? x 1]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
       n
   <dbl>
1 104681

### 006
レシート明細データ（df_receipt）から売上⽇（sales_ymd）、顧客（customer_id）、
商品コード（product_cd）、売上数量（quantity）、売上⾦額（amount）の順に列を指定
し、以下の全ての条件を満たすデータを抽出せよ。
- 顧客ID（customer_id）が"CS018205000001"
- 売上⾦額（amount）が1,000以上または売上数量（quantity）が5以上

In [7]:
# 006
receipt_tbl %>%
filter(amount >= 1000 || quantity >=5) %>%
filter(customer_id == 'CS018205000001') %>%
select(sales_ymd, customer_id, product_cd, quantity, amount) %>% 
head

# Source:   lazy query [?? x 5]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  sales_ymd customer_id    product_cd quantity amount
      <int> <chr>          <chr>         <int>  <int>
1  20180911 CS018205000001 P071401012        1   2200
2  20180414 CS018205000001 P060104007        6    600
3  20170614 CS018205000001 P050206001        5    990
4  20190226 CS018205000001 P071401020        1   2200
5  20180911 CS018205000001 P071401005        1   1100

### 015
顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベッ
トのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項⽬抽出し、10件表⽰せよ。

In [8]:
library(stringr)

customer_tbl %>%
filter(str_detect(status_cd, "^[A-F].*[1-9]$")) %>%
select(customer_id, status_cd) %>%
head

# Source:   lazy query [?? x 2]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  customer_id    status_cd   
  <chr>          <chr>       
1 CS011215000048 C-20100421-9
2 CS022513000105 A-20091115-7
3 CS001515000096 A-20100724-7
4 CS013615000053 B-20100329-6
5 CS020412000161 B-20081021-3
6 CS001215000097 A-20100211-2

### 016
店舗データ（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [9]:
store = tbl(con, 'store')
store %>%
filter(str_detect(tel_no, "^\\d{3}-\\d{3}-\\d{4}$")) %>%
head

# Source:   lazy query [?? x 10]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  store_cd store_name   prefecture_cd prefecture address     address_kana tel_no
  <chr>    <chr>                <int> <chr>      <chr>       <chr>        <chr> 
1 S12014   千草台店                12 千葉県     千葉県千葉… チバケンチ…  043-1…
2 S13002   国分寺店                13 東京都     東京都国分… トウキョウ…  042-1…
3 S14010   菊名店                  14 神奈川県   神奈川県横… カナガワケ…  045-1…
4 S14033   阿久和店                14 神奈川県   神奈川県横… カナガワケ…  045-1…
5 S14036   相模原中央店            14 神奈川県   神奈川県相… カナガワケ…  042-1…
6 S14040   長津田店                14 神奈川県   神奈川県横… カナガワケ…  045-1…
# … with 3 more variables: longitude <dbl>, latitude <dbl>, floor_area <dbl>

### 019
レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [10]:
re = tbl(con,'receipt')
re %>%
select(customer_id, amount) %>%
mutate(ranking = min_rank(desc(amount))) %>%
arrange(ranking) %>%
head

# Source:     lazy query [?? x 3]
# Database:   postgres 12.0.11 [@localhost:5432/knock100]
# Ordered by: ranking
  customer_id    amount ranking
  <chr>           <int>   <dbl>
1 CS011415000006  10925       1
2 ZZ000000000000   6800       2
3 CS028605000002   5780       3
4 ZZ000000000000   5480       4
5 CS015515000034   5480       4
6 ZZ000000000000   5480       4

### 020
レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [11]:
re %>%
select(customer_id,amount) %>%
mutate(ranking = row_number(desc(amount))) %>%
arrange(ranking) %>%
head

# Source:     lazy query [?? x 3]
# Database:   postgres 12.0.11 [@localhost:5432/knock100]
# Ordered by: ranking
  customer_id    amount ranking
  <chr>           <int>   <dbl>
1 CS011415000006  10925       1
2 ZZ000000000000   6800       2
3 CS028605000002   5780       3
4 ZZ000000000000   5480       4
5 CS015515000034   5480       5
6 ZZ000000000000   5480       6

### 026
レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年⽉⽇（sales_ymd）と古い売上年⽉⽇を求め、両者が異なるデータを10件表⽰せよ。

In [12]:
re %>%
group_by(customer_id) %>%
summarise(latest_ymd = max(sales_ymd, na.rm=TRUE), oldest_ymd = min(sales_ymd,na.rm=TRUE)) %>%
filter(latest_ymd != oldest_ymd) %>%
head

# Source:   lazy query [?? x 3]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  customer_id    latest_ymd oldest_ymd
  <chr>               <int>      <int>
1 CS029212000033   20180621   20170318
2 CS007515000119   20190511   20170201
3 CS034515000123   20190708   20170527
4 CS026414000014   20190720   20170718
5 CS010515000082   20181204   20180518
6 CS019315000045   20170920   20170423

### 028
レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上額
（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [13]:
re %>%
group_by(store_cd) %>%
summarise(amount = median(amount, na.rm = TRUE)) %>%
arrange(desc(amount)) %>%
head

# Source:     lazy query [?? x 2]
# Database:   postgres 12.0.11 [@localhost:5432/knock100]
# Ordered by: desc(amount)
  store_cd amount
  <chr>     <dbl>
1 S13052      190
2 S14010      188
3 S14050      185
4 S14040      180
5 S13003      180
6 S13018      180

### 029
レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求め、10件表示させよ。

In [21]:
re %>%
group_by(store_cd, product_cd) %>%
summarise(count = n(), .groups = 'drop_last') %>%
filter(count == max(count, na.rm = TRUE)) %>%
head

# Source:   lazy query [?? x 3]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
# Groups:   store_cd
  store_cd product_cd count
  <chr>    <chr>      <dbl>
1 S12007   P060303001    72
2 S12013   P060303001   107
3 S12014   P060303001    65
4 S12029   P060303001    92
5 S12030   P060303001   115
6 S13001   P060303001    67

In [20]:
re %>%
group_by(store_cd, product_cd) %>%
summarise(count = n()) %>%
filter(count == max(count)) %>%
head

`summarise()` has grouped output by 'store_cd'. You can override using the `.groups` argument.


# Source:   lazy query [?? x 3]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
# Groups:   store_cd
  store_cd product_cd count
  <chr>    <chr>      <dbl>
1 S12007   P060303001    72
2 S12013   P060303001   107
3 S12014   P060303001    65
4 S12029   P060303001    92
5 S12030   P060303001   115
6 S13001   P060303001    67

### R-031:
レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標準偏差を計算し、降順で5件表示せよ。

In [15]:
receipt_tbl %>%
group_by(store_cd) %>%
summarise(sd = sd(amount, na.rm = TRUE)) %>%
arrange(desc(sd)) %>%
head

# Source:     lazy query [?? x 2]
# Database:   postgres 12.0.11 [@localhost:5432/knock100]
# Ordered by: desc(sd)
  store_cd    sd
  <chr>    <dbl>
1 S13052    665.
2 S14011    554.
3 S14034    545.
4 S13001    544.
5 S13015    544.
6 S13035    527.

### R-033:
レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [16]:
re %>%
group_by(store_cd) %>%
summarise(total = mean(amount, na.rm = TRUE)) %>%
filter(total>=330) %>%
head

# Source:   lazy query [?? x 2]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  store_cd total
  <chr>    <dbl>
1 S13052    403.
2 S13019    330.
3 S13003    351.
4 S14045    330.
5 S13004    331.
6 S13001    348.

### R-036:
レシート明細データ（df_receipt）と店舗データ（df_store）を内部結合し、レシート明細データの全項目と店舗データの店舗名（store_name）を10件表示せよ。

In [17]:
# store %>% head
 inner_join(re, store, by = 'store_cd')

# Source:   lazy query [?? x 18]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
   sales_ymd sales_epoch store_cd receipt_no receipt_sub_no customer_id   
       <int>       <int> <chr>         <int>          <int> <chr>         
 1  20181103  1541203200 S14006          112              1 CS006214000001
 2  20181118  1542499200 S13008         1132              2 CS008415000097
 3  20170712  1499817600 S14028         1102              1 CS028414000014
 4  20190205  1549324800 S14042         1132              1 ZZ000000000000
 5  20180821  1534809600 S14025         1102              2 CS025415000050
 6  20190605  1559692800 S13003         1112              1 CS003515000195
 7  20181205  1543968000 S14024         1102              2 CS024514000042
 8  20190922  1569110400 S14040         1102              1 CS040415000178
 9  20170504  1493856000 S13020         1112              2 ZZ000000000000
10  20191010  1570665600 S14027         1102              1 CS027514000015
# … with mo

### R-038:
顧客データ（df_customer）とレシート明細データ（df_receipt）から、顧客ごとの売上金額合計を求め、10件表示せよ。ただし、売上実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [18]:
cu = tbl(con,'customer')

top20cu <- cu %>% filter(gender_cd == 1 && !str_detect(customer_id, '^Z')) %>%
left_join(re, by = 'customer_id') %>%
group_by(customer_id) %>%
summarise(amount_total = sum(ifelse(is.null(amount),0,amount))) %>%
arrange(desc(amount_total))

# show
top20cu %>% head

Warning message:
“Missing values are always removed in SQL.
Use `SUM(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”


# Source:     lazy query [?? x 2]
# Database:   postgres 12.0.11 [@localhost:5432/knock100]
# Ordered by: desc(amount_total)
  customer_id    amount_total
  <chr>                 <dbl>
1 CS017415000097        23086
2 CS015415000185        20153
3 CS031414000051        19202
4 CS028415000007        19127
5 CS010214000010        18585
6 CS006515000023        18372

### R-039:
レシート明細データ（df_receipt）から、売上日数の多い顧客の上位20件を抽出したデータと、売上金額合計の多い顧客の上位20件を抽出したデータをそれぞれ作成し、さらにその2つを完全外部結合せよ。ただし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [19]:
re %>%
filter(!str_detect(customer_id, '^Z') && !is.na(sales_ymd)) %>%
group_by(customer_id) %>%
summarise(n_day = n_distinct(sales_ymd)) %>%
arrange(desc(n_day)) %>%
head(20) %>%
full_join(top20cu, by = 'customer_id')

# Source:     lazy query [?? x 3]
# Database:   postgres 12.0.11 [@localhost:5432/knock100]
# Ordered by: desc(n_day)
   customer_id    n_day amount_total
   <chr>          <dbl>        <dbl>
 1 CS001112000009    NA            0
 2 CS001112000019    NA            0
 3 CS001112000021    NA            0
 4 CS001112000023    NA            0
 5 CS001112000024    NA            0
 6 CS001112000029    NA            0
 7 CS001112000030    NA            0
 8 CS001113000004     1         1298
 9 CS001113000010    NA            0
10 CS001114000005     2          626
# … with more rows